**Целью** данной курсовой работы является дообучение и оптимизация предобученной модели машинного перевода с использованием методов машинного обучения для автоматического перевода текстов с английского языка на русский.</div>

**Задачи:**
1. Исследование предметной области: Определить основные концепции и методы машинного перевода, включая предобученные модели, путем обзора литературы.
2. Подготовка данных: Провести процесс сбора, очистки и предварительной обработки данных для обучения и тестирования модели машинного перевода.
3. Выбор датасета: Выбрать датасет для использования в исследовании.
4. Предварительная обработка данных: Применить необходимые методы предварительной обработки данных.
5. Дообучение модели: Применить модель(и) для дообучения предварительно обученной модели машинного перевода на выбранных данных.
6. Сопоставление данных: Оценить соответствие между исходными текстами и их переводами для оценки качества работы модели.
7. Метрики: Поиск метрики качества перевода и сравнение различных моделей.
8. Цикл обучения.
9. Подготовка всего к обучению: Подготовить окружение для обучения модели, включая настройку вычислительных ресурсов и инфраструктуры.
10. Цикл обучения: Провести обучение модели на подготовленных данных с использованием выбранных методов и алгоритмов.
11. Использование дообученной модели: Оценить производительность дообученной модели и проанализировать результаты.

**Актуальность:** С ростом интернациональных коммуникаций и расширением онлайн-платформ становится все более важным обеспечить быстрый и качественный перевод контента на различные языки. Компании, особенно те, которые работают в сфере онлайн-бизнеса и социальных сетей, сталкиваются с необходимостью локализации своих продуктов и контента для различных регионов и языковых аудиторий.

В этом контексте, дообучение предобученных моделей машинного перевода представляет собой эффективный инструмент для автоматизации этого процесса. Благодаря развитию технологий машинного обучения, таких как нейронные сети и глубокое обучение, становится возможным создание более точных и гибких систем перевода, способных эффективно справляться с разнообразными типами текстов и языковыми особенностями.

Дообучение и оптимизация предобученных моделей машинного перевода остается востребованной задачей в контексте современного мирового онлайн-пространства.

# 1. **Подготовка данных**

In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.3 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")

**В рамках данной работы для анализа и дообучения модели были использованы данные из датасета "kde4", доступного на платформе Hugging Face. Hugging Face - это общедоступная платформа, где специалисты в области машинного обучения могут совместно работать над различными моделями, наборами данных и приложениями. Датасет "kde4" был добавлен одним из пользователей этой платформы. Для наших целей мы взяли первые 3500 строк из этого датасета. Этот подход позволил нам сосредоточиться на конкретном подмножестве данных, что облегчило процесс обучения и анализа.**

In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="ru")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
raw_datasets['train'] = raw_datasets['train'].select(range(3500))

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 3500
    })
})

**Был проведен анализ тематического моделирования на наборе данных, содержащем тексты переводов. Он использует два основных инструмента: TF-IDF векторизатор и модель Latent Dirichlet Allocation (LDA).**

1. TF-IDF векторизатор: TF-IDF (Term Frequency-Inverse Document Frequency) - это статистическая мера, используемая для оценки важности слова в документе, который является частью коллекции документов. Он увеличивает пропорционально числу раз, когда слово появляется в документе, но смещается на частоту слова в корпусе, что помогает скорректировать факт, что некоторые слова встречаются более часто в целом.

2. Модель LDA: LDA - это генеративная статистическая модель, которая позволяет объяснить наборы наблюдений с помощью групп неявных переменных. В контексте обработки текста, LDA обычно используется для тематического моделирования, где каждый документ считается смесью различных тем, и каждая тема предполагает определенное распределение слов.

**В данном коде, сначала из набора данных извлекаются тексты переводов. Затем эти тексты преобразуются в TF-IDF векторы. Эти векторы затем используются для обучения модели LDA.**

**После обучения модели, код выводит 10 ключевых слов для каждой темы. Это делается путем сортировки вероятностей слов для каждой темы и выбора 10 слов с наибольшей вероятностью.**

**В результате, мы выявили основные темы и соответствующие ключевые слова в текстовых данных, что может оказаться полезным для понимания содержания и организации имеющихся данных.**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

translations = [example['translation']['en'] for example in raw_datasets['train']]

tfidf_vectorizer = TfidfVectorizer(max_features=1000, max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(translations)

lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(tfidf)

for idx, topic in enumerate(lda.components_):
    print(f"Набор {idx}:", end=' ')
    print([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Набор 0: ['kuickshow', 'mail', 'selection', 'insert', 'files', 'object', 'tree', 'project', 'color', 'templates']
Набор 1: ['documentation', 'zoom', 'use', 'toolbars', 'section', 'midi', 'working', 'tools', 'layout', 'view']
Набор 2: ['network', 'irc', 'command', 'tab', 'nick', 'nickname', 'list', 'screen', 'server', 'channel']
Набор 3: ['check', 'gallery', 'user', 'size', 'kdeaddons', 'support', 'amarok', 'installation', 'credits', 'image']
Набор 4: ['kxsldbg', 'msg', 'menu', 'shift', 'left', 'xsldbg', 'help', 'configure', 'settings', 'com']
Набор 5: ['url', 'using', 'open', 'dialog', 'icon', 'quantaplus', 'render', 'toolbar', 'click', 'window']
Набор 6: ['open', 'new', 'save', 'kde', 'net', 'edit', 'org', 'file', 'ctrl', 'roles_of_translators']
Набор 7: ['debugger', 'actions', 'grid', 'isdale', 'xml', 'keith', 'type', 'plugins', 'konqueror', 'tag']
Набор 8: ['camera', 'variables', 'robert', 'views', 'konversation', 'breakpoints', 'debug', 'povray', 'default', 'usage']
Набор 9: ['hand

**Одна из особенностей этого датасета, наполненного техническими терминами в области компьютерных наук, заключается в том, что все они полностью переведены на русский язык. Однако зачастую при разговоре русскоговорящих "айтишников"  большинство специфических для компьютерных наук слов не переводится, а остается на английском.**

**Однако в современном диалоге можно услышать в русской речи непереведенные тех. термины. Например, слово "plugins" вполне может встретиться в русском предложении как плагин (независимо компилируемый программный модуль), особенно в техническом разговоре, однако, в данном датасете оно переведено как более правильное: “модуль”.**

**У нас имеется 3500 пар слов/словосочетаний/предложений, но они находятся только в train выборке, поэтому необходимо создать собственный набор данных для test.**

In [7]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 3150
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 350
    })
})

In [8]:
split_datasets["validation"] = split_datasets.pop("test")

Пример содержимого скаченного датасета:

In [9]:
split_datasets["train"][328]["translation"]

{'en': 'plugins', 'ru': 'модули'}

In [10]:
split_datasets["train"][5]["translation"]

{'en': 'Compilation and Installation', 'ru': 'Сборка и установка'}

In [11]:
split_datasets["train"][327]["translation"]

{'en': 'You can select a song to play in the Collection Manager, or by using the combo box over the karaoke text.',
 'ru': 'Вы можете выбрать песню, которую хотите прослушать, с помощью Менеджера коллекций или с помощью выпадающего списка над текстом караоке.'}

In [ ]:
!pip install tf-keras

**Мы использовали предобученную модель машинного перевода от Helsinki-NLP (Helsinki-NLP/opus-mt-...-... — это модель обработки естественного языка (NLP), реализованная в библиотеке Transformer, состоящая из энкодера, декодера и шести слоев нейронов, специально обученную для перевода с английского на русский язык. Эта модель была выбрана из-за ее высокой производительности и способности обрабатывать различные типы текстов.
Модель была загружена и инициализирована с помощью функции pipeline из библиотеки transformers (трансформер -  это такой вид нейросетевой архитектуры, который хорошо подходит для обработки последовательностей данных. В контексте обработки естественного языка, трансформеры позволяют модели учитывать контекст каждого слова в предложении, что делает их особенно эффективными для таких задач, как перевод, генерация текста и классификация текста. Трансформеры могут обрабатывать все слова в предложении одновременно, что делает их более быстрыми и эффективными.)**

**Использование этой предобученной модели позволило нам существенно ускорить процесс обучения и улучшить качество перевода. Благодаря этому мы смогли сосредоточиться на анализе результатов и интерпретации полученных данных, а не на длительном и трудоемком процессе обучения модели с нуля.**

In [13]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
translator = pipeline("translation", model=model_checkpoint)

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

In [14]:
translator("plugins")

[{'translation_text': 'Подсказки'}]

# 2. **Предварительная обработка данных**

**Даллее было необходимо преобразовать все тексты в наборы идентификаторов токенов, чтобы модель могла понять их смысл. Для этой задачи нам понадобится токенизация как входных данных, так и целевых. Наша первая задача - создать объект tokenizer. Как отмечалось ранее, мы будем использовать предварительно обученную модель Helsinki-NLP en - ru.**

In [15]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

**Подготовка наших данных довольно проста. Нужно помнить только об одном: необходимо убедиться, что токенизатор обрабатывает целевые значения на выходном языке (здесь - русском).**

In [16]:
en_sentence = split_datasets["train"][327]["translation"]["en"]
ru_sentence = split_datasets["train"][327]["translation"]["ru"]

inputs = tokenizer(en_sentence, text_target=ru_sentence)
inputs

{'input_ids': [103, 127, 18829, 13, 10924, 9, 2110, 10, 4, 32833, 31169, 2, 64, 39, 2146, 4, 12256, 10198, 9131, 361, 4, 2583, 22838, 24806, 1701, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [394, 3769, 12244, 6635, 2666, 2, 1583, 4873, 48343, 2, 16, 1830, 21653, 30, 16036, 41, 5295, 5882, 11276, 74, 16, 1830, 128, 5697, 14621, 11201, 730, 18931, 14991, 771, 30, 3, 0]}

**Последним шагом будет определение функции предварительной обработки, которую мы будем применять к датасетам:**

In [17]:
max_length = 128 # одинаковая максимальная длина для наших входов и выходов (так как тексты довольно коротки)
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["ru"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [18]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/3150 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

**В результате tokenized_datasets будет содержать токенизированные версии исходных наборов данных, готовые к использованию для обучения модели.**

# 3. **Дообучение модели с Keras**

**Прежде всего, нам нужна актуальная модель, которую нужно дообучить. Keras - это высокоуровневый API для глубокого обучения, написанный на Python. Он способен работать поверх более низкоуровневых фреймворков глубокого обучения, таких как TensorFlow, Theano и CNTK. Keras был разработан с целью облегчения быстрого прототипирования и исследований в области глубокого обучения.** 

**API AutoModel в библиотеке Hugging Face Transformers - это удобный интерфейс, который позволяет автоматически выбирать правильную модель для задачи на основе предобученного чекпоинта.**

**При обучении TFAutoModelForSeq2SeqLM, мы передали имя предобученной модели. API AutoModel затем автоматически определила тип модели на основе этого чекпоинта и возвратила соответствующий класс модели.**

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM
# model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

# 4. **Сопоставление данных**

**В нашей задаче инструмент DataCollatorForSeq2Seq собирает и форматирует данные, чтобы они могли быть эффективно использованы в процессе обучения.**


In [21]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

**Для обработки динамического батча требуется коллатор данных, способный обрабатывать дополнения (padding). Для подготовки данных в динамическом батче, включая дополнение меток до максимальной длины, используется DataCollatorForSeq2Seq. Этот коллатор данных также генерирует входные идентификаторы декодера, сдвинутые версии меток с начальным токеном. Чтобы использовать DataCollatorForSeq2Seq, необходимо указать объект модели. В результате анализа на нескольких примерах подтверждается корректная обработка меток и создание сдвинутых версий для декодера. Таким образом, DataCollatorForSeq2Seq обеспечивает готовность данных для обучения модели.**

**Батч в контексте машинного обучения - это подмножество данных, которое используется для обучения модели в течение одной итерации.
Когда мы говорим о "батче данных", мы обычно имеем в виду группу примеров обучающих данных, которые обрабатываются одновременно. Размер батча - это количество примеров в одном батче.**

Чтобы протестировать его на нескольких примерах, мы просто вызываем его на списке примеров из нашего токинезированного обучающего набора:

In [22]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [26]:
batch["labels"][1]

<tf.Tensor: shape=(25,), dtype=int32, numpy=
array([  246, 43106,   246, 14922,  7779,    67,     0,  -100,  -100,
        -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
        -100,  -100,  -100,  -100,  -100,  -100,  -100], dtype=int32)>

In [27]:
batch["decoder_input_ids"][1]

<tf.Tensor: shape=(25,), dtype=int32, numpy=
array([62517,   246, 43106,   246, 14922,  7779,    67,     0, 62517,
       62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517,
       62517, 62517, 62517, 62517, 62517, 62517, 62517], dtype=int32)>

In [28]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[1003, 22124, 222, 7105, 17796, 84, 46431, 5033, 16504, 49092, 30, 22993, 100, 2745, 6, 31, 2583, 1026, 99, 91, 402, 649, 12, 3, 0]
[246, 43106, 246, 14922, 7779, 67, 0]


**Теперь мы можем использовать этот data_collator для преобразования каждого из наших датасетов в tf.data.Dataset, готовый к обучению:**

In [29]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

# 5. **Метрики**

**Оценка BLEU принимает уже существующие идеально хорошие переводы как эталонный перевод и сравнивает выходные данные машинного перевода (кандидата) с этим эталоном. Оценка не измеряет разборчивость или грамматическую правильность сгенерированных моделью результатов, но использует статистические правила, чтобы гарантировать, что все слова в сгенерированных результатах также встречаются в целевых.**

**Один из недостатков BLEU заключается в том, что она предполагает, что текст уже прошел токенизацию, что затрудняет сравнение оценок между моделями, использующими различные токенизаторы. Поэтому сегодня для сравнения моделей перевода чаще всего используется метрика SacreBLEU, которая устраняет этот недостаток**

In [30]:
!pip install sacrebleu

In [31]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [32]:
import evaluate
metric = evaluate.load("sacrebleu")

In [33]:
predictions = [
    "Я пошел в парк и увидел великолепное озеро."
]
references = [
    [
        "Я пошел в парк и увидел великолепное море."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 75.06238537503395,
 'counts': [8, 6, 5, 4],
 'totals': [9, 8, 7, 6],
 'precisions': [88.88888888888889, 75.0, 71.42857142857143, 66.66666666666667],
 'bp': 1.0,
 'sys_len': 9,
 'ref_len': 9}

In [34]:
predictions = ["Жили были жили были."]
references = [
    [
        "Жили были дед и баба."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 19.3576934939088,
 'counts': [3, 1, 0, 0],
 'totals': [5, 4, 3, 2],
 'precisions': [60.0, 25.0, 16.666666666666668, 12.5],
 'bp': 0.8187307530779819,
 'sys_len': 5,
 'ref_len': 6}

**Оценка может варьироваться от 0 до 100, причем чем больше, тем лучше.**

В этом коде был создан новый объект generation_data_collator, используя функцию DataCollatorForSeq2Seq из библиотеки Transformers. Этот объект был настроен так, чтобы дополнять данные до размера, кратного 128, что оптимизирует работу с XLA.

Затем был подготовлен набор данных для генерации с использованием этого коллатора и модели. Набор данных был подготовлен без перемешивания и с размером батча 8.

Была определена функция generate_with_xla, которая использует модель для генерации вывода на основе входных данных батча. Эта функция была оптимизирована для использования с XLA.

Затем была определена функция compute_metrics, которая вычисляет метрики для валидационного набора данных. Она проходит через каждый батч в наборе данных, генерирует предсказания с помощью функции generate_with_xla, декодирует предсказания и метки, и затем вычисляет BLEU-оценку для предсказаний и меток. Результаты затем возвращаются в виде словаря.

In [35]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
)


def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=128,
    )


def compute_metrics():
    all_preds = []
    all_labels = []

    for batch, labels in tqdm(tf_generate_dataset):
        predictions = generate_with_xla(batch)
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels.numpy()
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)

    result = metric.compute(predictions=all_preds, references=all_labels)
    return {"bleu": result["score"]}

XLA, или Accelerated Linear Algebra, это компилятор оптимизации, разработанный Google для улучшения производительности машинного обучения. Он работает с TensorFlow, одной из наиболее популярных библиотек для машинного обучения.

XLA улучшает производительность машинного обучения, оптимизируя вычислительные графы TensorFlow. Он анализирует структуру графа, определяет, какие операции можно объединить или упростить, и затем генерирует эффективный код для выполнения этих операций.


# 6. **Дообучение модели**

Результаты модели без какого-либо обучения:

In [ ]:
print(compute_metrics())

100%|██████████| 44/44 [1:24:21<00:00, 115.04s/it]

{'bleu': 13.574768727081302}


In [7]:
bleu1 = 13.575

Был создан оптимизатор для обучения модели с помощью функции create_optimizer из библиотеки Transformers.

Затем модель была скомпилирована с использованием этого оптимизатора.

Был создан callback для сохранения модели после каждой эпохи обучения. Этот callback сохраняет только веса модели, сохраняет только лучшую модель (с наименьшим значением потерь на валидационном наборе данных), и сохраняет модель в указанном файле.

Наконец, модель была обучена на обучающем наборе данных, с валидационным набором данных для проверки производительности модели после каждой эпохи, и с callback для сохранения модели после каждой эпохи. Обучение продолжалось в течение указанного количества эпох (1, однако время обучения заняло примено 1.5 часа)

In [37]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# Количество шагов обучения - это количество примеров в датасете, разделенное на размер батча, затем умноженное на общее количество эпох.
num_epochs = 1
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Создание callback для сохранения модели после каждой эпохи
checkpoint_callback = ModelCheckpoint(
    filepath="C:\\Users\\CoolerMaster\\kde4-en-to-ru/checkpoint.ckpt",
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[checkpoint_callback],
    epochs=num_epochs,
)


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
98/98 [==============================] - ETA: 0s - loss: 2.0438 
Epoch 1: val_loss improved from inf to 1.76269, saving model to C:\Users\CoolerMaster\kde4-en-to-ru/checkpoint.ckpt
98/98 [==============================] - 4480s 45s/step - loss: 2.0438 - val_loss: 1.7627


In [ ]:
print(compute_metrics())

100%|██████████| 44/44 [1:35:21<00:00, 130.04s/it]

{'bleu': 18.4759763535845}


In [4]:
bleu2 = 18.476

# 7. **Использование дообученной модели**

In [41]:
from transformers import pipeline
model_checkpoint = "C:\\Users\\CoolerMaster\\kde4-en-to-ru/checkpoint.ckpt"
translator = pipeline("translation", model=model_checkpoint)
translator("plugins")

[{'translation_text': 'модули'}]


Как и ожидалось, наша предварительно обученная модель адаптировала свои знания к корпусу, на котором мы ее дообучили, и вместо того, чтобы оставить слово "подсказки" в покое, она теперь переводит его как "модули" из дообученной части модели.

# 8. **Выводы**

In [14]:
data = {
    'BLEU предобученной модели': [bleu1],
    'BLEU дообученной предобученной модели': [bleu2]
}
pd.DataFrame(data)

,BLEU предобученной модели,BLEU дообученной предобученной модели
0,13.575,18.476


В результате проведенных экспериментов было обнаружено, что предобученная модель без дополнительного обучения показала BLEU-оценку равную 13.58. BLEU-оценка - это метрика, которая оценивает качество машинного перевода, сравнивая его с переводом, выполненным человеком. Чем выше BLEU-оценка, тем ближе перевод машины к переводу человека.

Однако, после дополнительного обучения на специфических данных, предобученная модель NLP от университета Хельсинки показала значительное улучшение, достигнув BLEU-оценки 18.48. Это указывает на то, что дополнительное обучение помогло модели лучше понять контекст и нюансы языка, на котором проводился перевод, что привело к улучшению качества перевода.

Эти результаты подчеркивают важность дополнительного обучения в области машинного перевода. Даже если модель была предварительно обучена на большом наборе данных, дополнительное обучение на специфических данных может значительно улучшить ее производительность. Это особенно важно в случаях, когда перевод требуется для специфического контекста или домена, такого как медицинские тексты, юридические документы или техническая терменология.

# 9. **Библиография**

1. Белоусова, Ю. (2022, 29 марта). Как мы обучили нейросети переводить тексты для сервиса «Облако MTS». Habr. https://habr.com/ru/companies/cloud_mts/articles/770202/
2. Hugging Face. (n.d.). Глава 1.4: Модель машинного перевода (Machine Translation). Введение в обработку естественного языка (NLP) с Hugging Face. https://huggingface.co/learn/nlp-course/ru/chapter1/4
3. Hugging Face. (n.d.). Helsinki-NLP/opus-mt-ru-en: English-Russian Machine Translation. Hugging Face Models Hub. https://huggingface.co/Helsinki-NLP/opus-mt-ru-en
4. Hugging Face. (n.d.). Модели машинного обучения для задач перевода (Translation). Hugging Face Documentation. https://huggingface.co/docs/transformers/tasks/translation
5. Назаров, К. (2019, 25 июля). Transformer: что это такое и как оно работает. Habr. https://habr.com/ru/articles/470706/
6. Sanjeev, K. (2019, 13 мая). Machine Translation with Transformer in Python. GeeksforGeeks. https://translated.turbopages.org/proxy_u/en-ru.ru.5cb4e200-663cda43-c62b9c1e-74722d776562/https/www.geeksforgeeks.org/machine-translation-with-transformer-in-python/